In [3]:
# ==============================
# Install required packages
# ==============================
!pip install langchain langchain-openai langchain-community pypdf python-docx

# ==============================
# Imports
# ==============================
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_community.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader

# ==============================
# Set your OpenAI API key
# ==============================
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

# ==============================
# Step 1: Define Extraction Schema
# ==============================
schemas = [
    ResponseSchema(name="name", description="Full name of the candidate"),
    ResponseSchema(name="age", description="Age in numbers"),
    ResponseSchema(name="gender", description="Gender of the candidate"),
    ResponseSchema(name="location", description="City or location"),
    ResponseSchema(name="email", description="Email ID"),
    ResponseSchema(name="phone", description="Phone number"),
    ResponseSchema(name="qualification", description="Highest qualification"),
    ResponseSchema(name="experience_years", description="Years of work experience"),
    ResponseSchema(name="skills", description="List of skills"),
    ResponseSchema(name="summary", description="Short professional summary")
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

# ==============================
# Step 2: Prompt Template
# ==============================
prompt = PromptTemplate(
    template="""
    Extract candidate details from this resume text:

    {resume_text}

    {format_instructions}
    """,
    input_variables=["resume_text"],
    partial_variables={"format_instructions": format_instructions},
)

# ==============================
# Step 3: Resume Loader
# ==============================
def load_resume(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(file_path)
    else:
        raise ValueError("Unsupported file format")
    return loader.load()[0].page_content

# ==============================
# Step 4: LLM Setup (OpenAI)
# ==============================
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def extract_resume_data(resume_text):
    prompt_text = prompt.format(resume_text=resume_text)
    response = llm.predict(prompt_text)
    return parser.parse(response)

# ==============================
# Step 5: Process Multiple Resumes
# ==============================
def process_resumes(resume_folder):
    all_candidates = []
    for file in os.listdir(resume_folder):
        if not (file.endswith(".pdf") or file.endswith(".docx")):
            continue
        path = os.path.join(resume_folder, file)
        print(f"Processing {file} ...")
        text = load_resume(path)
        candidate_data = extract_resume_data(text)
        all_candidates.append(candidate_data)

    df = pd.DataFrame(all_candidates)
    df.to_csv("extracted_candidates.csv", index=False)
    print("✅ Data saved to extracted_candidates.csv")
    return d


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.7 MB/s eta 0:00:00
